In [2]:
import configparser
import os
from joblib import dump, load
import json
from tqdm import tqdm
from helpers.helper_functions import *
from helpers.helper_classes import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from joblib import dump, load

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

c:\Users\caspa\Desktop\data-mining-techniques-vu\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [91]:
# Read config.ini file
config = configparser.ConfigParser()
config.read('src/config.ini')
# os.chdir(config['PATH']['ROOT_DIR'])

# # Load data
df = pd.read_parquet(config['PATH']['DATA_DIR'] + '/training_set.parquet', engine = 'fastparquet')
df_test = pd.read_parquet(config['PATH']['DATA_DIR'] + '/test_set.parquet', engine = 'fastparquet')

# mini df for testing quickly 
df_mini = df[df['srch_id'] < 10000]
# df.drop('Unnamed: 0', axis=1, inplace=True)

# time to datetime
# df['time'] = pd.to_datetime(df['time'])


In [92]:
def construct_target(row):
    if row['booking_bool'] == 1:
        return 5
    elif row['click_bool'] == 1:
        return 1
    else:
        return 0

def binary_target(row):
    if row['booking_bool'] == 1:
        return 1
    else:
        return 0
    
def construct_target_df(df):
    df.loc[:, 'target'] = np.zeros(len(df))
    df.loc[df['click_bool'] == 1, 'target'] = 1
    df.loc[df['booking_bool'] == 1, 'target'] = 5
    return df['target']

def drop_cols(df, cols):
    return df.drop(cols, axis=1)

In [93]:
df_1 = df_mini.copy()
df_t = df_test.copy()

print('copied')
# Construct target and drop leaky columns

construct_target_df(df_1)
# df_1['grades'] = construct_target_df(df_1)

def drop_cols(df, cols):
    # check which cols are in dff
    cols = [col for col in cols if col in df.columns]
    return df.drop(cols, axis=1)

def construct_datetime(df):
    df_out = df.copy()
    df_out['date_time'] = pd.to_datetime(df_out['date_time'])
    df_out['month'] = df_out['date_time'].dt.month
    df_out['day'] = df_out['date_time'].dt.day
    df_out['hour'] = df_out['date_time'].dt.hour
    return df_out

def preprocess(df, train_set = True):
    leaky_cols = ['click_bool', 'booking_bool', 'gross_bookings_usd', 'position', 'date_time', 'target']

    df_out = df.copy()
    df_out = construct_datetime(df_out)
    if train_set:
        df_out['grades'] = construct_target_df(df_out)
    df_out = drop_cols(df_out, leaky_cols)
    df_out.fillna(-1, inplace=True)
    return df_out

df_1 = preprocess(df_1)
df_t = preprocess(df_t, train_set=False)
df_t
# # Construct datetime features
# df_1 = construct_datetime(df_1)
# df_t = construct_datetime(df_t)

# df_1.loc[:, 'grades'] = df_1['target']

# # Drop leaky columns



# leaky_cols = ['click_bool', 'booking_bool', 'gross_bookings_usd', 'position', 'date_time', 'target']
# df_1 = drop_cols(df_1, leaky_cols)


# ## Test: Drop all columns with NaNs
# # df_1 = df_1.dropna(axis=1)
# df_1.fillna(0, inplace=True)



# # Construct dummy cols
# dummy_cols = ['prop_country_id', 'visitor_location_country_id', 'srch_destination_id']
# df_1 = pd.get_dummies(df_1, columns=dummy_cols, drop_first=True)

# df_1

copied


,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,month,day,hour
0,1,24,216,-1.0,-1.0,219,3180,3,4.5,1,2.94,0.0691,5.03,119.00,0,19222,1,10,2,0,1,0,-1.0,-1.0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,2,15
1,1,24,216,-1.0,-1.0,219,5543,3,4.5,1,2.64,0.0843,4.93,118.00,0,19222,1,10,2,0,1,0,-1.0,-1.0,0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,2,15
2,1,24,216,-1.0,-1.0,219,14142,2,3.5,1,2.71,0.0556,4.16,49.00,0,19222,1,10,2,0,1,0,-1.0,-1.0,0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,0.0,10.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,2,15
3,1,24,216,-1.0,-1.0,219,22393,3,4.5,1,2.40,0.0561,5.03,143.00,0,19222,1,10,2,0,1,0,-1.0,-1.0,0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,2,15
4,1,24,216,-1.0,-1.0,219,24194,3,4.5,1,2.94,0.2090,4.72,79.00,0,19222,1,10,2,0,1,0,-1.0,-1.0,0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,2,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4959178,332787,24,216,-1.0,-1.0,117,32019,4,3.5,0,2.48,0.0551,4.53,66.07,0,19246,2,7,1,0,1,0,-1.0,-1.0,0,-1.0,-1.0,-1.0,1.0,0.0,22.0,1.0,0.0,127.0,-1.0,0.0,27.0,1.0,0.0,22.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,5,21,11
4959179,332787,24,216,-1.0,-1.0,117,33959,4,3.0,1,2.20,0.3344,4.39,67.10,0,19246,2,7,1,0,1,0,-1.0,-1.0,0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,16.0,1.0,0.0,22.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,5,21,11
4959180,332787,24,216,-1.0,-1.0,117,35240,4,0.0,0,1.79,-1.0000,4.64,73.91,0,19246,2,7,1,0,1,0,-1.0,-1.0,0,-1.0,-1.0,-1.0,1.0,0.0,55.0,0.0,0.0,-1.0,0.0,0.0,16.0,0.0,0.0,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,5,21,11
4959181,332787,24,216,-1.0,-1.0,117,94437,4,0.0,0,2.94,0.0928,4.64,66.07,0,19246,2,7,1,0,1,0,-1.0,-1.0,0,-1.0,-1.0,-1.0,1.0,0.0,43.0,1.0,0.0,43.0,-1.0,0.0,12.0,-1.0,0.0,12.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,5,21,11


In [95]:
# Perform logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import roc_auc_score

def train_test_split(df, target_str, test_size=.2):
    splitter = GroupShuffleSplit(test_size=test_size, n_splits=2, random_state = 7)
    split = splitter.split(df, groups=df['srch_id'])
    train_inds, test_inds = next(split)

    df_ideal = df.iloc[test_inds].copy().sort_values(by=['srch_id', 'grades'], ascending=[True, False], inplace=False)

    X = df_1.drop(['grades'], axis=1)
    y = df_1[target_str]
    X_train, X_test, y_train, y_test, test_ideal = X.iloc[train_inds], X.iloc[test_inds], y.iloc[train_inds], y.iloc[test_inds], df_ideal, 


    return X_train, X_test, y_train, y_test, test_ideal[['srch_id', 'prop_id', 'grades']]

def construct_pred_ideal(df_in, df_ideal, y_pred):
    df = df_in.copy()
    df['pred_grades'] = y_pred
    df = df.sort_values(by=['srch_id', 'pred_grades'], ascending=[True, False], inplace=False)

    # Merge grades from ideal on srch_id and prop_id
    df = df.merge(df_ideal, on=['srch_id', 'prop_id'], how='left')

    # Return srch_id, prop_id and pred_grades
    return df[['srch_id', 'prop_id', 'pred_grades', 'grades']]

def construct_pred_submission(df_in, y_pred):
    df = df_in.copy()
    df['pred_grades'] = y_pred
    df = df.sort_values(by=['srch_id', 'pred_grades'], ascending=[True, False], inplace=False)

    # Return srch_id, prop_id and pred_grades
    return df[['srch_id', 'prop_id']]

def constructs_predictions(model, data, ideal_df = None):
    y_pred = model.predict_proba(data)
    pred_grades = y_pred @ [0, 1, 5]

    if ideal_df is not None:
        pred_df = construct_pred_ideal(data, test_ideal, pred_grades)
    else:
        pred_df = construct_pred_submission(data, pred_grades)
    return pred_df


def calc_NDCG(df_ideal, df_pred, k = 5):
    # Group by 5
    df_ideal = df_ideal.groupby('srch_id').head(k)
    df_pred = df_pred.groupby('srch_id').head(k)

    assert df_ideal.shape[0] % k == 0
    assert df_pred.shape[0] % k == 0
    
    # Get grades matrices
    ideal_grades = df_ideal['grades'].values.reshape(int(df_ideal.shape[0] / k), k)
    pred_grades = df_pred['grades'].values.reshape(int(df_pred.shape[0] / k), k)

    discount_vec = [1/np.log2(i+2) for i in range(k)]

    # Calculate NDCG
    NDCG = (pred_grades @ discount_vec).sum() / (ideal_grades @ discount_vec).sum()

    return NDCG

X_train, X_test, y_train, y_test, test_ideal = train_test_split(df_1, 'grades')

In [90]:

# Train KNN classifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score


# Random forest
from sklearn.ensemble import RandomForestClassifier
print('Training RF')
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
rf.fit(X_train, y_train)

# XGBoost
print('Training XGB')
import xgboost as xgb
y_train_xgb = y_train.astype(int)
y_train_xgb[y_train == 5] = 2
xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
xgb_model.fit(X_train, y_train_xgb)

pred_ideal_rf = constructs_predictions(rf, X_test, ideal_df=test_ideal)
pred_xgb = constructs_predictions(xgb_model, X_test, ideal_df=test_ideal)
pred_random = construct_pred_ideal(X_test, test_ideal, np.random.rand(len(X_test)))

Training RF
Training XGB


KeyboardInterrupt: 

In [96]:
# Optimize XGB with optuna
import optuna
from functools import partial

def objective(trial, X_train, y_train, X_test, test_ideal):
    y_train_xgb = y_train.astype(int)
    y_train_xgb[y_train == 5] = 2

    params = {
        "objective": "multi:softprob",
        "random_state": 42,
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 1),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-4, 1e-1, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-4, 1e-1, log=True),
    }

    xgb_model = xgb.XGBClassifier(**params)
    xgb_model.fit(X_train, y_train_xgb)

    pred_xgb = constructs_predictions(xgb_model, X_test, ideal_df=test_ideal)
    ndcg = calc_NDCG(test_ideal, pred_xgb)

    return ndcg

print("Training XGB")
# Assuming you have defined X_train, y_train, X_test, and test_ideal before this point.

# Wrap the objective function with the input data
objective_with_data = partial(objective, X_train=X_train, y_train=y_train, X_test=X_test, test_ideal=test_ideal)

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction="maximize")
study.optimize(objective_with_data, n_trials=20)



[I 2023-05-09 13:56:31,830] A new study created in memory with name: no-name-3a7cbff6-7ed5-4fc1-8c20-9778f5cce14a


Training XGB


[I 2023-05-09 13:58:02,635] Trial 0 finished with value: 0.3422170675116014 and parameters: {'n_estimators': 218, 'max_depth': 10, 'learning_rate': 0.002795642578981349, 'subsample': 0.8932459525721343, 'colsample_bytree': 0.6546014752508442, 'gamma': 0.4545479889258107, 'reg_alpha': 0.0006735472057143736, 'reg_lambda': 0.05659086785788689}. Best is trial 0 with value: 0.3422170675116014.
[I 2023-05-09 13:58:34,110] Trial 1 finished with value: 0.3296238751532763 and parameters: {'n_estimators': 122, 'max_depth': 5, 'learning_rate': 0.00020205115375924383, 'subsample': 0.6995347755906247, 'colsample_bytree': 0.9885228465832642, 'gamma': 0.19381601429279216, 'reg_alpha': 0.03803815623242628, 'reg_lambda': 0.00015357257740569215}. Best is trial 0 with value: 0.3422170675116014.
[I 2023-05-09 13:59:37,760] Trial 2 finished with value: 0.3368665578611753 and parameters: {'n_estimators': 472, 'max_depth': 4, 'learning_rate': 0.004549507912707027, 'subsample': 0.9923851580887791, 'colsample_

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2], got [0 1 5]

In [97]:
y_train_xgb = y_train.astype(int)
y_train_xgb[y_train == 5] = 2

# Get the best hyperparameters and train the final model
best_params = study.best_params
xgb_model_optimized = xgb.XGBClassifier(objective="multi:softprob", random_state=42, **best_params)
xgb_model_optimized.fit(X_train, y_train_xgb)

# Evaluate the optimized model
pred_xgb_optimized = constructs_predictions(xgb_model_optimized, X_test, ideal_df=test_ideal)
pred_xgb_submission = constructs_predictions(xgb_model_optimized, df_t)
print(f"XGB Optimized: {calc_NDCG(test_ideal, pred_xgb_optimized)}")

XGB Optimized: 0.3586960449767132


## Evaluation


In [87]:
print(f"RF: {calc_NDCG(test_ideal, pred_ideal_rf)}\n,XGB: {calc_NDCG(test_ideal, pred_xgb)},\nRandom: {calc_NDCG(test_ideal, pred_random)}")

RF: 0.28572611476380494
,XGB: 0.3413833342709407,
Random: 0.16416903670552785


In [98]:
# pred_submission.to_csv(config['PATH']['DATA_DIR'] + '/submission_RF.csv', index=False)
pred_xgb_submission.to_csv(config['PATH']['DATA_DIR'] + '/submission_XGB.csv', index=False)

In [99]:
pred_xgb_submission

,srch_id,prop_id
23,1,99484
9,1,54937
12,1,61934
5,1,28181
13,1,63894
...,...,...
4959177,332787,29018
4959178,332787,32019
4959181,332787,94437
4959182,332787,99509
